In [ ]:
 ! pip install -q kaggle

In [ ]:
from google.colab import files

In [ ]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"unnati675657","key":"9e7ed7a08a4527bc56758e7b3d878209"}'}

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

ref                                                      title                                     size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------------  ---------------------------------------  -----  -------------------  -------------  ---------  ---------------  
syedanwarafridi/vehicle-sales-data                       Vehicle Sales Data                        19MB  2024-02-21 20:16:17          18009        321  1.0              
sudarshan24byte/online-food-dataset                      Online Food Dataset                        3KB  2024-03-02 18:50:30           7819        136  0.9411765        
murilozangari/jobs-and-salaries-in-data-field-2024       Jobs and Salaries in Data field 2024     127KB  2024-03-18 18:15:53           1268         24  0.9411765        
ayushparwal2026/cars-dataset                             Cars Dataset                             169KB  2024-03-11 16:23:49           1781         30

In [ ]:
! kaggle competitions download -c 'deepfake-detection-challenge'

100% 4.12G/4.13G [00:48<00:00, 135MB/s]
100% 4.13G/4.13G [00:48<00:00, 91.9MB/s]


In [ ]:
! mkdir train

In [ ]:
! unzip deepfake-detection-challenge.zip -d train

Archive:  deepfake-detection-challenge.zip
  inflating: train/sample_submission.csv  
  inflating: train/test_videos/aassnaulhq.mp4  
  inflating: train/test_videos/aayfryxljh.mp4  
  inflating: train/test_videos/acazlolrpz.mp4  
  inflating: train/test_videos/adohdulfwb.mp4  
  inflating: train/test_videos/ahjnxtiamx.mp4  
  inflating: train/test_videos/ajiyrjfyzp.mp4  
  inflating: train/test_videos/aktnlyqpah.mp4  
  inflating: train/test_videos/alrtntfxtd.mp4  
  inflating: train/test_videos/aomqqjipcp.mp4  
  inflating: train/test_videos/apedduehoy.mp4  
  inflating: train/test_videos/apvzjkvnwn.mp4  
  inflating: train/test_videos/aqrsylrzgi.mp4  
  inflating: train/test_videos/axfhbpkdlc.mp4  
  inflating: train/test_videos/ayipraspbn.mp4  
  inflating: train/test_videos/bcbqxhziqz.mp4  
  inflating: train/test_videos/bcvheslzrq.mp4  
  inflating: train/test_videos/bdshuoldwx.mp4  
  inflating: train/test_videos/bfdopzvxbi.mp4  
  inflating: train/test_videos/bfjsthfhbd.mp4  
  

In [ ]:
TRAIN_DIR = '/content/train/train_sample_videos/'
TMP_DIR = '/content/train/tmp/'
ZIP_NAME = 'dfdc_train_faces_sample.zip'
METADATA_PATH = TRAIN_DIR + 'metadata.json'

SCALE = 0.25
N_FRAMES = None

In [ ]:
!pip install facenet-pytorch > /dev/null 2>&1
!apt install zip > /dev/null 2>&1

In [ ]:
import os
import glob
import json
import torch
import cv2
from PIL import Image
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from facenet_pytorch import MTCNN

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Running on device: {device}')

Running on device: cpu


DEFINE FACE-EXTRACTER

In [ ]:
class FaceExtractor:
    def __init__(self, detector, n_frames=None, resize=None):
        """
        Parameters:
            n_frames {int} -- Total number of frames to load. These will be evenly spaced
                throughout the video. If not specified (i.e., None), all frames will be loaded.
                (default: {None})
            resize {float} -- Fraction by which to resize frames from original prior to face
                detection. A value less than 1 results in downsampling and a value greater than
                1 result in upsampling. (default: {None})
        """

        self.detector = detector
        self.n_frames = n_frames
        self.resize = resize

    def __call__(self, filename, save_dir):
        """Load frames from an MP4 video, detect faces and save the results.

        Parameters:
            filename {str} -- Path to video.
            save_dir {str} -- The directory where results are saved.
        """

        # Create video reader and find length
        v_cap = cv2.VideoCapture(filename)
        v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Pick 'n_frames' evenly spaced frames to sample
        if self.n_frames is None:
            sample = np.arange(0, v_len)
        else:
            sample = np.linspace(0, v_len - 1, self.n_frames).astype(int)

        # Loop through frames
        for j in range(v_len):
            success = v_cap.grab()
            if j in sample:
                # Load frame
                success, frame = v_cap.retrieve()
                if not success:
                    continue
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = Image.fromarray(frame)

                # Resize frame to desired size
                if self.resize is not None:
                    frame = frame.resize([int(d * self.resize) for d in frame.size])

                save_path = os.path.join(save_dir, f'{j}.png')

                self.detector([frame], save_path=save_path)

        v_cap.release()

In [ ]:
with open(METADATA_PATH, 'r') as f:
    metadata = json.load(f)

In [ ]:
train_df = pd.DataFrame(
    [
        (video_file, metadata[video_file]['label'], metadata[video_file]['split'], metadata[video_file]['original'] if 'original' in metadata[video_file].keys() else '')
        for video_file in metadata.keys()
    ],
    columns=['filename', 'label', 'split', 'original']
)

train_df.head()

,filename,label,split,original
0,aagfhgtpmv.mp4,FAKE,train,vudstovrck.mp4
1,aapnvogymq.mp4,FAKE,train,jdubbvfswz.mp4
2,abarnvbtwb.mp4,REAL,train,None
3,abofeumbvv.mp4,FAKE,train,atvmxvwyns.mp4
4,abqwwspghj.mp4,FAKE,train,qzimuostzz.mp4


In [ ]:
# Load face detector
face_detector = MTCNN(margin=14, keep_all=True, factor=0.5, device=device).eval()

In [ ]:
# Define face extractor
face_extractor = FaceExtractor(detector=face_detector, n_frames=N_FRAMES, resize=SCALE)

In [ ]:
# Get the paths of all train videos
all_train_videos = glob.glob(os.path.join(TRAIN_DIR, '*.mp4'))

In [ ]:
!mkdir -p $TMP_DIR

In [ ]:
import random
num_videos_to_process = 50
selected_videos = random.sample(all_train_videos, num_videos_to_process)

with torch.no_grad():
    for path in tqdm(selected_videos, desc="Processing videos"):
        file_name = os.path.basename(path)
        save_dir = os.path.join(TMP_DIR, os.path.splitext(file_name)[0])

        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        # Detect all faces appear in the video and save them.
        face_extractor(path, save_dir)

Processing videos:   0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
cd $TMP_DIR

/content/dataset/tmp


In [ ]:
train_df.to_csv('metadata.csv', index=False)